In [1]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/My Drive/plant_pathology_kaggle")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

sam_sub = pd.read_csv('sample_submission.csv')
train_csv = pd.read_csv('train.csv')
test_csv = pd.read_csv('test.csv')

In [3]:
train_csv.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [4]:
def get_class(row): 
  if row['multiple_diseases'] == 1: 
    return 'multiple_diseases' 
  elif row['rust'] == 1: 
    return 'rust' 
  elif row['scab'] == 1: 
    return 'scab' 
  else: 
    return 'healthy' 
    
train_csv["label"] = train_csv.apply(get_class, axis=1) 
#train_csv.head()
temp = pd.factorize(train_csv["label"])
train_csv["label"] = temp[0]

train_csv.head()

,image_id,healthy,multiple_diseases,rust,scab,label
0,Train_0,0,0,0,1,0
1,Train_1,0,1,0,0,1
2,Train_2,1,0,0,0,2
3,Train_3,0,0,1,0,3
4,Train_4,1,0,0,0,2


In [5]:
reference_map = {"scab" : 0, "multiple_diseases" : 1, "healthy" : 2, "rust" : 3}
y = train_csv["label"]

from keras.utils import to_categorical
y = to_categorical(y, 4)
y

Using TensorFlow backend.


array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)

In [6]:
test_csv.head()

,image_id
0,Test_0
1,Test_1
2,Test_2
3,Test_3
4,Test_4


In [0]:
path = '/content/drive/My Drive/plant_pathology_kaggle/images/'
image_list = os.listdir(path)

train_images = train_csv["image_id"]
test_images = test_csv["image_id"]

train_images = np.asarray(train_images) + ".jpg"
test_images = np.asarray(test_images) + ".jpg"

In [8]:
train_images

array(['Train_0.jpg', 'Train_1.jpg', 'Train_2.jpg', ..., 'Train_1818.jpg',
       'Train_1819.jpg', 'Train_1820.jpg'], dtype=object)

In [9]:
image_height = 150
image_width = 150
channels = 3

import cv2
xtrain = np.zeros((len(y), image_height, image_width, channels))
xtest = np.zeros((len(y), image_height, image_width, channels))

for i in range(len(y)):
  imgtrain = train_images[i]
  imgtest = test_images[i]
  #path = '/content/drive/My Drive/plant_pathology_kaggle/images/' + img
  img = cv2.imread('/content/drive/My Drive/plant_pathology_kaggle/images/'+imgtrain)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (image_height, image_width))
  xtrain[i] = img
  img = cv2.imread('/content/drive/My Drive/plant_pathology_kaggle/images/'+imgtest)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (image_height, image_width))
  xtest[i] = img

"""
for i in range(len(y)):
  img = test_images[i]
  path = '/content/drive/My Drive/plant_pathology_kaggle/images/' + img
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (image_height, image_width))
  xtest[i] = img
"""

"\nfor i in range(len(y)):\n  img = test_images[i]\n  path = '/content/drive/My Drive/plant_pathology_kaggle/images/' + img\n  img = cv2.imread(path)\n  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\n  img = cv2.resize(img, (image_height, image_width))\n  xtest[i] = img\n"

In [10]:
print(xtrain.shape)
print(xtest.shape)

(1821, 256, 256, 3)
(1821, 256, 256, 3)


In [0]:
xtrain /= 255
xtest /= 255

In [0]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state = 267)

x_res, y_res = ros.fit_resample(xtrain.reshape((-1, image_height * image_width * channels)), y)
x_res = x_res.reshape((-1, image_height, image_width, channels))

x_res.shape, y_res.sum(axis=0)

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xcv, ytrain, ycv = train_test_split(x_res, y_res, test_size = 0.1, random_state = 267)

####Thats all basic pre-procesing.
Things to try next : 
  #iii. transfer learning
  #iv. without reshaping 

In [13]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras import models

model = models.Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = "same", 
                 activation = "relu", input_shape = (image_height, image_width, channels)))
model.add(MaxPool2D(pool_size=(2,2)))
#model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = "same", 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
#model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.25))


model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = "same", 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = "same", 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

#model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = "same", 
#                 activation ='relu'))
#model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
#model.add(Dropout(0.25))

#model.add(Conv2D(filters = 512, kernel_size = (3,3),padding = "same", 
#                 activation ='relu'))
#model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
#model.add(Dropout(0.5))
model.add(Dense(4, activation = "softmax"))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
from keras.optimizers import RMSprop
opti = RMSprop(lr = 1e-3)
model.compile(optimizer = opti, loss = "categorical_crossentropy", metrics = ["accuracy"])

from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, 
                                            verbose=1, factor=0.5, min_lr=0.00001)

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = "/content/res_aug.h5", 
                             save_best_only = True, monitor = "val_loss",
                             verbose = 1)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range = 45, width_shift_range = 0.2, height_shift_range = 0.2,
                             zoom_range = 0.2, horizontal_flip = True, vertical_flip = True)

datagen.fit(xtrain)

train_size = len(ytrain)

In [0]:
history = model.fit_generator(datagen.flow(x = xtrain, y = ytrain, batch_size = 32), 
                    epochs = 50, steps_per_epoch = len(y)//32, 
                    validation_data = (xcv, ycv), 
                    callbacks = [learning_rate_reduction, checkpoint])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/50





56/56 [==============================] - 34s 613ms/step - loss: 10.2787 - acc: 0.3526 - val_loss: 11.2739 - val_acc: 0.3005

Epoch 00001: val_loss improved from inf to 11.27386, saving model to /content/aug_train2.h5
Epoch 2/50
56/56 [==============================] - 24s 422ms/step - loss: 10.6434 - acc: 0.3397 - val_loss: 11.2739 - val_acc: 0.3005

Epoch 00002: val_loss did not improve from 11.27386
Epoch 3/50
56/56 [==============================] - 25s 444ms/step - loss: 10.5355 - acc: 0.3464 - val_loss: 11.2739 - val_acc: 0.3005

Epoch 00003: val_loss did not improve from 11.27386
Epoch 4/50
56/56 [==============================] - 24s 425ms/step - loss: 10.5047 - acc: 0.3483 - val_loss: 11.2739 - val_acc: 0.3005

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00004: val_loss did not improve from 11.27386
Epoch 5/50
56/56 [=======

In [0]:
import matplotlib.pyplot as plt
#plt.subplots(2,1)
#plt.subplot(1, 2, 1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.title("Model accuracy")
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

In [0]:
from keras.models import load_model
net = load_model("/content/res_aug.h5")

predictions = net.predict(xtest)
temp = predictions
predictions = np.argmax(predictions, axis = 1)

In [0]:
sam_sub["healthy"].values[:] = 0
sam_sub["multiple_diseases"].values[:] = 0
sam_sub["rust"].values[:] = 0
sam_sub["scab"].values[:] = 0

for i in range(len(predictions)):
  if predictions[i] == 0:
    sam_sub["scab"].values[i] = 1
  elif predictions[i] == 1:
    sam_sub["multiple_diseases"].values[i] = 1
  elif predictions[i] == 2:
    sam_sub["healthy"].values[i] = 1
  elif predictions[i] == 3:
    sam_sub["rust"].values[i] = 1

sam_sub.head()

In [0]:
sam_sub.to_csv("/content/res_aug.csv", index = False)